# Teams Over Time

This notebook explains the 'horizontal' slice of how sentiment analysis depicts a particular team's gamethreads over multiple weeks.

## Imports

In [1]:
# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/MiscDataProjects/collaboration/RedditTextAnalysis


In [2]:
# by including this autoreload command, we only need to re-import if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

In [3]:
import pandas as pd
import sqlite3
import altair as alt

## Query Comments For Fan-Level Analysis

In [4]:
with sqlite3.connect("nfl_gamethreads.db") as conn:

    # pull in all comments from weeks 4-9, 2021
    comments_df = pd.read_sql_query('''SELECT comments.comment_id, comments.author, comments.body, comments.author_flair, 
                                              comments.upvotes, comments.utc_time, comments.polarity, comments.subjectivity,
                                              gamethreads.week, gamethreads.season
                                    FROM comments 
                                    INNER JOIN gamethreads ON comments.submission_id = gamethreads.submission_id 
                                    WHERE gamethreads.week <= 9''', conn)

In [5]:
comments_df

,comment_id,author,body,author_flair,upvotes,utc_time,polarity,subjectivity,week,season
0,hej5m66,Piano_Fingerbanger,Boo both teams! \n\nLet's get a tie!,buccaneers,2,1.632786e+09,0.000000,0.000000,3,2021
1,hej5n9d,None,[deleted],none,5,1.632786e+09,0.000000,0.000000,3,2021
2,hej5n9g,TheBigSm0ke,Good measuring stick game for Hurts. Let’s see...,none,1,1.632786e+09,0.150000,0.500000,3,2021
3,hej5ner,harshalax31,Hopefully the streak of amazing prime time gam...,49ers,2,1.632786e+09,0.600000,0.900000,3,2021
4,hej5nmk,Saitsu,"Oh I'm ready, BRING ON THE TOXICITY!\n\nAs for...",none,3,1.632786e+09,-0.183333,0.366667,3,2021
...,...,...,...,...,...,...,...,...,...,...
636998,hjgrea1,Lieutenant_Meeper,Goddamn the most recent Super Bowl win really ...,broncos,1,1.636146e+09,0.438889,0.452778,9,2021
636999,hjgwm8t,andrewfnluck,It’s all in fun. I would prefer a couple of co...,colts,1,1.636148e+09,0.400000,0.433333,9,2021
637000,hjh8nqv,None,[deleted],none,1,1.636152e+09,0.000000,0.000000,9,2021
637001,hjh9rbz,Zivmovic,I forgot the scientists that have made vaccine...,giants,1,1.636153e+09,0.338542,1.000000,9,2021


### Aggregate by Flair Across All Games

This represents the entire fandom across all gamethreads on a weekly basis.

In [6]:
week_sum = comments_df[['polarity','subjectivity','week', 'author_flair']].groupby(['week', 'author_flair']).mean().reset_index()

In [51]:
alt.Chart(data=week_sum).mark_line().encode(
    alt.X("week:N"),
    alt.Color("author_flair"),
    y=("polarity")
).properties(
    width=800,
    height=800
)

alt.Chart(...)

### Aggregate by a team's gamethreads over time

In [52]:
with sqlite3.connect("nfl_gamethreads.db") as conn:

    # pull in all comments from weeks 4-9, 2021
    comments_df = pd.read_sql_query('''SELECT comments.comment_id, comments.author, comments.body, comments.author_flair, 
                                              comments.upvotes, comments.utc_time, comments.polarity, comments.subjectivity,
                                              gamethreads.week, gamethreads.season, gamethreads.home_team, gamethreads.away_team
                                    FROM comments 
                                    INNER JOIN gamethreads ON comments.submission_id = gamethreads.submission_id 
                                    WHERE gamethreads.week <= 9''', conn)

In [53]:
comments_df

,comment_id,author,body,author_flair,upvotes,utc_time,polarity,subjectivity,week,season,home_team,away_team
0,hej5m66,Piano_Fingerbanger,Boo both teams! \n\nLet's get a tie!,buccaneers,2,1.632786e+09,0.000000,0.000000,3,2021,Dallas Cowboys,Philadelphia Eagles
1,hej5n9d,None,[deleted],none,5,1.632786e+09,0.000000,0.000000,3,2021,Dallas Cowboys,Philadelphia Eagles
2,hej5n9g,TheBigSm0ke,Good measuring stick game for Hurts. Let’s see...,none,1,1.632786e+09,0.150000,0.500000,3,2021,Dallas Cowboys,Philadelphia Eagles
3,hej5ner,harshalax31,Hopefully the streak of amazing prime time gam...,49ers,2,1.632786e+09,0.600000,0.900000,3,2021,Dallas Cowboys,Philadelphia Eagles
4,hej5nmk,Saitsu,"Oh I'm ready, BRING ON THE TOXICITY!\n\nAs for...",none,3,1.632786e+09,-0.183333,0.366667,3,2021,Dallas Cowboys,Philadelphia Eagles
...,...,...,...,...,...,...,...,...,...,...,...,...
636998,hjgrea1,Lieutenant_Meeper,Goddamn the most recent Super Bowl win really ...,broncos,1,1.636146e+09,0.438889,0.452778,9,2021,Indianapolis Colts,New York Jets
636999,hjgwm8t,andrewfnluck,It’s all in fun. I would prefer a couple of co...,colts,1,1.636148e+09,0.400000,0.433333,9,2021,Indianapolis Colts,New York Jets
637000,hjh8nqv,None,[deleted],none,1,1.636152e+09,0.000000,0.000000,9,2021,Indianapolis Colts,New York Jets
637001,hjh9rbz,Zivmovic,I forgot the scientists that have made vaccine...,giants,1,1.636153e+09,0.338542,1.000000,9,2021,Indianapolis Colts,New York Jets


In [72]:
titans = comments_df[(comments_df['home_team'] == 'Tennessee Titans') | (comments_df['away_team'] == "Tennessee Titans")].copy()

titans_by_week = titans[['polarity','subjectivity','week', 'author_flair']].groupby(['week']).mean().reset_index()
titans_by_week['team'] = 'Titans'

In [73]:
saints = comments_df[(comments_df['home_team'] == 'New Orleans Saints') | (comments_df['away_team'] == "New Orleans Saints")].copy()

saints_by_week = saints[['polarity','subjectivity','week', 'author_flair']].groupby(['week']).mean().reset_index()
saints_by_week['team'] = 'Saints'

In [78]:
all_by_week = comments_df[['polarity','subjectivity','week', 'author_flair']].groupby(['week']).mean().reset_index()
all_by_week['team'] = 'All'

In [79]:
ten_no = pd.concat([titans_by_week, saints_by_week, all_by_week])

In [81]:
alt.Chart(data=ten_no).mark_line().encode(
    alt.X("week:N"),
    alt.Color("team:N"),
    y=("polarity")
).properties(
    width=800,
    height=800
)

alt.Chart(...)